## Import packages

In [13]:
import pandas as pd
from CFModel import CFModel

## Define constants


In [14]:
RATINGS_CSV_FILE = './input/rating_result.csv'
USERS_CSV_FILE = './input/user_result.csv'
RECIPE_CSV_FILE = './input/recipe_result.csv'
MODEL_WEIGHTS_FILE = 'recipe_weights.h5'
K_FACTORS = 120
TEST_USER = 10

## Load Recipe data

In [15]:
ratings = pd.read_csv(RATINGS_CSV_FILE, sep=',', encoding='utf-8', usecols=['userid', 'recipeid', 'rating'])
max_userid = ratings['userid'].drop_duplicates().max() + 1
max_recipeid = ratings['recipeid'].drop_duplicates().max() + 1
print(len(ratings), 'ratings loaded.')

(9999, 'ratings loaded.')


In [16]:
users = pd.read_csv(USERS_CSV_FILE, sep=',', encoding='utf-8', usecols=['userid', 'nickname'])
print(len(users), 'descriptions of', max_userid, 'users loaded.')

(75061, 'descriptions of', 39, 'users loaded.')


In [17]:
recipes = pd.read_csv(RECIPE_CSV_FILE, sep=',', encoding='utf-8', usecols=['recipeid'])#, 'title', 'categories'])
print(len(recipes), 'descriptions of', max_recipeid, 'recipes loaded.')

(5101, 'descriptions of', 5017, 'recipes loaded.')


## Make recommendations for a given user

In [18]:
trained_model = CFModel(max_userid, max_recipeid, K_FACTORS)

In [19]:
trained_model.load_weights(MODEL_WEIGHTS_FILE)

In [20]:
users[users['userid'] == TEST_USER]

,userid,nickname
10,10,dreamlove1


In [21]:
def predict_rating(userid, recipeid):
    return trained_model.rate(userid, recipeid)

In [22]:
user_ratings = ratings[ratings['userid'] == TEST_USER][['userid', 'recipeid', 'rating']]

In [23]:
user_ratings['prediction'] = user_ratings.apply(lambda x: predict_rating(TEST_USER, x['recipeid']), axis=1)
user_ratings.sort_values(by='rating', 
                         ascending=False).merge(recipes, 
                                                on='recipeid', 
                                                how='inner',
                                                suffixes=['_u', '_m']
                                               ).head(10)

,userid,recipeid,rating,prediction
0,10,331,5,0.271748
1,10,159,5,0.300851
2,10,9,5,0.300829
3,10,20,5,0.237342
4,10,44,5,0.337480
5,10,70,5,0.242287
6,10,107,5,0.326903
7,10,129,5,0.291754
8,10,133,5,0.204415
9,10,135,5,0.293085


In [24]:
recommendations = ratings[ratings['recipeid'].isin(user_ratings['recipeid']) == False][['recipeid']].drop_duplicates()
recommendations['prediction'] = recommendations.apply(lambda x: predict_rating(TEST_USER, x['recipeid']), axis=1)
recommendations.sort_values(by='prediction',
                          ascending=False).merge(recipes,
                                                 on='recipeid',
                                                 how='inner',
                                                 suffixes=['_u', '_m']).head(10)

,recipeid,prediction
0,832,0.370515
1,972,0.353137
2,683,0.338949
3,762,0.338045
4,910,0.334884
5,1004,0.334834
6,688,0.334205
7,840,0.331273
8,1070,0.328433
9,850,0.324200
